In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import copy

tokenizer = AutoTokenizer.from_pretrained("rubra-ai/Meta-Llama-3-8B-Instruct")
num_expand = 4
base_model = AutoModelForCausalLM.from_pretrained("rubra-ai/Meta-Llama-3-8B-Instruct")
extra_layer1 = base_model.model.layers[8]
extra_layer2 = base_model.model.layers[17]
extra_layer3 = base_model.model.layers[26]
extra_layer4 = base_model.model.layers[35]

print(len(base_model.model.layers))


# TODO: we can load state_dict only
model_ft1_last_layer = copy.deepcopy(base_model.model.layers[-1].state_dict())
# model_ft2_last_layer = copy.deepcopy(AutoModelForCausalLM.from_pretrained("yingbei/tinyllama-ft2").model.layers[-1].state_dict())

def print_layer_checksum(layer, name="Layer"):
    # Compute and print a checksum of the layer's weights
    checksum = sum(p.sum().item() for p in layer.parameters())
    print(f"{name} checksum: {checksum}")

# def attach_extra_layer(base_model):
#     base_model.model.layers.append(extra_layer)
#     # print_layer_checksum(base_model.model.layers[-1])

def copy_last_layer_weights(base_model, layer_to_replace):
    base_model.model.layers[-1].load_state_dict(layer_to_replace)
    print_layer_checksum(base_model.model.layers[-1])


def detach_extra_layer(base_model):
    base_model.model.layers = base_model.model.layers[:-1]
    # for layers in base_model.model.layers:
        # reverse logic of block expansion
        # new_layers.append

import torch


Loading checkpoint shards:  25%|██▌       | 1/4 [00:09<00:28,  9.64s/it]

: 

In [2]:
# attach_extra_layer(base_model)
# copy_last_layer_weights(base_model, model_ft1_last_layer)
# detach_extra_layer(base_model)
# attach_extra_layer(base_model)
# copy_last_layer_weights(base_model, model_ft1_last_layer)


pipe = pipeline("text-generation", model=base_model, torch_dtype=torch.bfloat16, device_map="auto", tokenizer=tokenizer)
messages = [
    {
        "role": "system",
        "content": "",
    },
    {"role": "user", "content": "who is sanjay nadhavajhala?"},
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


IndexError: list index out of range